<a href="https://colab.research.google.com/github/vicente-gonzalez-ruiz/MCDWT-experimentos/blob/master/src/ejemplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [PRMC](https://github.com/Sistemas-Multimedia/PRMC)$(T=1,N=5)$
Calculamos la transformada directa e inversa de una iteración, a 5 imágenes. La descomposición espacial y temporal generada se puede ver [aquí](https://sistemas-multimedia.github.io/PRMC/#x1-20001r7).

In [ ]:
%matplotlib inline
from load_and_display import *

# The results depends on the wavelet (see http://wavelets.pybytes.com):
#wavelet = "haar"
wavelet = "bior3.5"
#wavelet = "rbio3.5"
#wavelet = "db5"
#wavelet = "db9"

 El vídeo usado es:

In [ ]:
show_seq("../sequences/stockholm/")

Los vídeos con los frames se almacenan en disco como imágnes PNG de 16 bits en el rango [0, 65535]. Esta tipo de formato se ha usado también para representar los coeficientes wavelet. Como estos pueden ser negativos, antes de ser almacenados a disco hay que sumarles 32768, y rezar para que su rango dinámico no exceda $2^{16}$ (cosa que rara vez ocurre si las imágenes de partida vienen a 8 bits/componente). Por tanto, tras leerlos de disco hay que restarles este valor. También ocurre que para visualizar los coeficientes hay que normalizarlos, porque no tienen por qué estar dentro del rango [0, 255]. El espacio de color usado para los frames es RGB, aunque en algunas fases del procesamiento haya que usar YCbCr. 

También podemos ver el movimiento en la secuencia restando imágenes consecutivas:

In [ ]:
show_motion("../sequences/stockholm/")

## Paso 1: $\mathtt{MDWT}(N=5)$

Aplicamos la 2D-DWT una iteración a cada una de las imágenes de la secuencia. La descomposición en subbandas generada es [esta](https://sistemas-multimedia.github.io/PRMC/#x1-10001r3).

In [ ]:
%%bash

# You must be in the ’src’ directory to run this.
rm -f /tmp/*.png  
cp ../sequences/stockholm/*.png /tmp  
 
# 1-iteration MDWT.  
python3 -O MDWT.py -p /tmp/  
 
# Show the length of the subbands.  
for i in /tmp/LL???.png; do ls -lh $i; done  
for i in /tmp/LH???.png; do ls -lh $i; done  
for i in /tmp/HL???.png; do ls -lh $i; done  
for i in /tmp/HH???.png; do ls -lh $i; done  

### Visualizamos la subbanda espacial $L$

La subbanda $L$ está compuesta por 5 subbandas espaciales $LL^1$ con una resolución mitad (en cada dimensión) de la original. [Aquí](https://sistemas-multimedia.github.io/PRMC/#x1-220009.5) se habla de la notación seguida para referenciar a las subbandas.

In [ ]:
show_seq("/tmp/LL")

In [ ]:
show_motion("/tmp/LL")

### Visualizamos la subbanda $H=\{LH, HL, HH\}$

In [ ]:
show_seq("/tmp/LH")

In [ ]:
show_seq("/tmp/HL")

In [ ]:
show_seq("/tmp/HH")

## Paso 2: $\mathtt{MCOLP}(T=1, N=5)$
Ahora compensamos la subbandas $F_1.H$ usando las subbandas $F_0.H$ y $F_2.H$ como referencia, y las subbandas $F_0.L$ y $F_2.L$ para estimar el movimiento. Como la DWT no es invariante al desplazamiento, en lugar de compensar en el dominio DWT compensamos en el dominio OLP.

In [ ]:
%%bash

# Hacemos una copia de las subbandas que vamos a compensar para luego poder comparar con lo que teníamos

rm -rf /tmp/tmp
mkdir /tmp/tmp
cp /tmp/LH001.png /tmp/tmp 
cp /tmp/HL001.png /tmp/tmp
cp /tmp/HH001.png /tmp/tmp
cp /tmp/LH003.png /tmp/tmp
cp /tmp/HL003.png /tmp/tmp
cp /tmp/HH003.png /tmp/tmp

In [ ]:
%%bash

predictor=1  
iterations=1  
 
# 1-iteration MCDWT.  
python3 -O MCOLP.py -P $predictor -p /tmp/ -T $iterations  

In [ ]:
%%bash

# Show the length of the original subbands and compensated subbands.
ls -lh /tmp/tmp/LH001.png
ls -lh /tmp/LH001.png
ls -lh /tmp/tmp/HL001.png
ls -lh /tmp/HL001.png
ls -lh /tmp/tmp/HH001.png
ls -lh /tmp/HH001.png
ls -lh /tmp/tmp/LH003.png
ls -lh /tmp/LH003.png
ls -lh /tmp/tmp/HL003.png
ls -lh /tmp/HL003.png
ls -lh /tmp/tmp/HH003.png
ls -lh /tmp/HH003.png

# Has changed in length any of them? Remember that a change in lenght  
# implies a change in content.  

Como se puede apreciar, la longitud de los ficheros `LH001` (antes: 698K, ahora: 701K), `HL001` (antes: 697K, ahora: 696K), `HH001` (antes: 494K, ahora: 516K), `LH003` (antes: 698K, ahora: 701K), `HL003` (antes: 696K, ahora: 696K), y `HH003` (antes: 494K, ahora: 517K), han cambiado (el resto de subbandas permanecen inalteradas). Estas son las subbandas que MCOLP *intenta* despojar de contenido visual mediante la compensación de movimiento. Como se puede ver, en algunos casos la entropía de orden N (suponiendo que la longitud de los archivos comprimidos con PNG sea proporcional a la entropía de orden N, donde N depende de cómo se está usando PNG) ha aumentado y en otros ha disminuido. Evidéntemente, lo mejor sería que siempre existiera una disminución, pero desde un punto de vista puramente visual, lossy, lo importante es que en dichas subbandas (las compensadas) desaparezcan las estructuras visualmente reconocibles. Esto significa que, si prescindiéramos completamente de dichas subbandas, un ser humano podría no darse cuenta de que esta información ya no está en el vídeo descomprimido.

Hay que considerar también otra fuente de ruido que puede perjudicar a la entropía de los residuos, y es que la implementación de la DWT que hemos usado ([PyWavelets](https://pywavelets.readthedocs.io/en/latest/)) trabaja en punto flotante, lo que significa que la transformada no es completamente reversible a causa del error de redondeo.

Esto es lo que ocurre visualmente ...

In [ ]:
show_img("/tmp/tmp/LH001") # Esta es la subbanda sin compensar

In [ ]:
show_img("/tmp/LH001") # Y esta la subbanda compensada

In [ ]:
show_img("/tmp/tmp/HL001") # Sin compensar

In [ ]:
show_img("/tmp/LH001") # Compensada

In [ ]:
show_img("/tmp/tmp/HH001") # Sin compensar

In [ ]:
show_img("/tmp/HH001") # Compensada

El parámetro $T=1$ que se le pasa al script $\mathtt{MCOLP}$ le indica que debe aplicar la [mariposa](https://sistemas-multimedia.github.io/PRMC/#x1-180009.1) una sola vez a todo el vídeo.

El parámetro `predictor=1` le indica a $\mathtt{MCOLP}$ que use como predicción la media aritmética de la predicción hacia delante y la predicción hacia detrás. Puede ser interesante saber que hay más posibilidades:

In [ ]:
%%bash
python3 MCOLP.py -h

Todas las predicciones están basadas en la [estimación del flujo óptico denso](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html#dense-optical-flow-in-opencv). Por tanto, en uno se los sentidos (hacia delante o hacia detrás), habría tantos vectores de movimiento como pixels se están compensando. Esta información la puede recrear el descompresor si reconstruye la subbanda LL sin pérdida (en este ejemplo la subbanda LL no se está comprimiendo con pérdida). En caso contrario, aparecería un error, que podríamos llamar de drift, y que se incrementa diádicamente para los niveles menor resolución temporal, acotado en cada GOP. En este ejemplo, el tamaño de GOP es 2 porque hemos iterado la MCOLP sólo una vez.

Para evitar el drift basta con usar en el compresor los mismos datos que debería tener el descompresor. Por ejemplo, si queremos usar HEVC intra para comprimir las subbandas $H$, deberíamos usar la versión descomprimida de $L$ (que reconstruirá el descompresor) para estimar el movimiento en el compresor (que estimaría el descompresor). Si usásemos JPEG2000 y capas de calidad, entonces para cada capa habría que realizar una estimación del movimiento, aunque seguramente muchos de los cálculos que se realizarían para la capa 1 servirían para la 2 y así, sucesivamente.

Un aspecto interesante de estimar el movimiento con las subbandas $L$ es que podemos medir la contribución de cada pixel de las imágenes vecinas a la predicción de la subbanda $L$ del frame que se está compensando, y con dicha medición, podemos estimar en qué medida cada frame vecino debería usarse para generar la (subbanda) predicción. Así, por ejemplo, como se explica [aquí](https://sistemas-multimedia.github.io/PRMC/#x1-2600010) (ver Eq. 5), podemos realizar una predicción ponderada estimando el error de predicción. Esto nos puede venir muy bien para cambios de escena, o para la aparición de nuevos objetos.

## Inverse PRMC($T=1, N=5$)
Reconstuímos la secuencia original (casi, PyWavelets use aritmética en punto flotante) a partir de la secuencia de subbandas.

In [ ]:
%%bash

predictor=1  
iterations=1

# Lets recover the original sequence ...  
rm /tmp/???.png  
 
# 1-iteration iMCDWT.  
python3 -O MCOLP.py -P $predictor -b -p /tmp/ -T $iterations  
 
# 1-iteration iMDWT.  
python3 -O MDWT.py -b -p /tmp/  

Mostramos la secuencia reconstruída:

In [ ]:
show_seq("/tmp/")

Y mostramos las diferencia entre las secuencias original y reconstruída:

In [ ]:
%%bash 
 
# Visualization of the residue.  
for i in {0..4}; do ii=$(printf "%03d" $i); bash ../tools/show_differences.sh -1 /tmp/$ii.png -2 ../sequences/stockholm/$ii.png -o /tmp/diff_$ii.png; done  

In [ ]:
show_seq("/tmp/diff_")

## Comprimiendo subbandas

Os digo lo que creo que podríamos hacer, dependiendo de la escalabilidad espacio/temporal requerida. Más adelante discutimos la escalabilidad por calidad.

### Comprimiendo directamente $L^0_0$ (vídeo original)
Si estamos comprimiendo $L^0_0$ es que no hemos aplicado PRMC. Podemos usar cualquier compresor de imágenes/vídeo, escalable o no, dependiendo de lo que necesitemos (aunque seguramente en modo no intra). Je, aquí me he reventado la cabeza pensando ...

### Comprimiendo $L^1_0$ y $H^1_{-1}$
Conseguir esta forma de escalabiliad únicamente temporal es un poco lioso con el código que hay desarrollado ahora mismo, pero posible si previamente a usar PRMC(1) interpolamos el vídeo con la DWT (usada para conseguir la OLP) para conseguir $L^0_{-1}$. De esta manera vamos a obtener las secuencias $L^1_0$ y $H^1_{-1}$ en las que las subbandas espaciales H (de ambas secuencias) son siempre 0, y por tanto se pueden ignorar. Cada frame (LL) de $H^1_{-1}$ debería comprimirse en modo intra y $L^1_0$ en modo no intra (dentro de cada GOP). El bit-allocation dentro de $L^1_0$ estaría controlado por el compresor de vídeo usado. Dentro del mismo GOP, el bit-allocation para las subbandas LL de $H^1_{-1}$ debería generar el mismo slope R/D que el que se ha conseguido para las subbandas LL de $L^1_0$.

### Comprimiendo $L^1_1$ y $H^1_0$
Este es el resultado de aplicar PRMC(1) a $L^0_0$, generándose 2 subbandas temporales: $L^1_1$ y $H^1_0$, la primera de resolución $Y/2\times X/2$ y la segunda proporcionando la resolución $Y\times X$. La subbanda temporal $L^1_1$ se debería comprimir en modo no intra, siendo $\lambda$ el R/D slope alcanzado, que si no me equivoco se podría calcular como (usando el MSE como medida de la distorsión) 
$$
    \lambda=\frac{\text{MSE}_\text{gray} - \text{MSE}}{\text{bit-rate}},
$$
donde $\text{MSE}_\text{gray}$ sería el resultado de comparar la secuencia "gris" (con todas sus componentes iguales a 128) con $L^1_1$, y $\text{MSE}$ resultaría de comparar $\hat{L}^1_1$ con $L^1_1$. $\text{bit-rate}$ es el bit-rate de la secuencia $\hat{L}^1_1$ comprimida.

Usando dicho $\lambda$, las subbandas espaciales de alta frecuencia de $H^1_0$ se deberían comprimir en 3 secuencias de vídeo independientes (una con las subbandas $LH$, otra con las subbandas $HL$ y otra con las subbandas $HH$) de manera que el slope total proporcionado fuera
$$
\lambda = \alpha\lambda'
$$
donde $\lambda'$ es el slope "local" generado, calculado como
$$
    \lambda'=\frac{\text{MSE}_\text{zero} - \text{MSE}}{\text{bit-rate}},
$$
donde $\text{MSE}_\text{zero}$ sería el resultado de comparar $\hat{H}^1_0$ con la secuencia "zero" (con todas sus componentes iguales a 0) y $\text{MSE}$ resultaría de comparar $\hat{H}^1_0$ con $H^1_0$. $\text{bit-rate}$ es el bit-rate total para las 3 secuencias comprimidas de $\hat{H}^1_0$.

## Cuantificando ... o comprimiendo con pérdida

La DWT, al usar "orthogonal basis functions" produce subbandas independientes, lo que significa que la contribución de una subbanda a la calidad de la reconstrucción no depende del contenido del resto de subbandas. Cuando aplicamos MCOLP la cosa cambia, porque en este caso existe una dependencia entre subbandas temporales. Analicemos algunos casos:

### Si no usamos MCOLP ...

Este caso ya está resuelto.

Las subbandas hay que 
... o comprimiendo, pero controlando la contribución de cada subbanda.

Dentro de un frame transformado, las subbandas no contribuyen en la misma proporción cuando se trata de reconstruir el frame (casi ninguna DWT es orthonormal ... lo escribo en inglés, porque en español me suena raro). Y dentro de una secuencia transformada, las subbandas temporales tampoco son igualmente importantes de cara a disminuir nuestro amado/odiado R/D tradeoff (las subbandas de menor frecuencia normalmente contribuyen más). De hecho en MCOLP, la transformada no es ni biorthogonal, lo que significa que lo que contribuye una subbanda temporal $H^x$ depende de lo que contribuyen las subbandas $H^{x+1}$, $H^{x+2}$ y por supuesto, la subbanda $L^y$ (siendo $y\geq x+2$).

Supongamos que aplicamos una MCOLP(3) a un vídeo, lo que significa que generamos 2 niveles de resolución espacial y 4 niveles de resolución temporal, generándose la subbandas $L^3_1, H^3_1, H^2_1, H^1_1, L^3_0, H^3_0, H^2_0, H^1_0$.

Aquí es donde está la madre del cordero (en controlar cuánto se cuantifican las diferentes subbandas) y donde veo yo el trabajo más duro. La solución más simple, y que en muchos casos funcionará más o menos bien, es suponer que MCOLP es al menos biorthogonal y por lo tanto, simplemente asignándole un peso adecuado a cada subbanda podemos cuantificar, y de ahí, pasar a comprimir.

Sin embargo, aunque el problema del rate-allocation parece complejo, podemos (creo yo) atacarlo suponiendo como en todo este proceso, que la reconstrucción del vídeo se va a producir por niveles de resolución (espaciales y temporales). Supongamos que queremos tener simplemente 2 resoluciones espaciales $L^0_0$ y $L^0_1$. Como primero vamos a recibir $L^0_1$ y el bit-rate lo va a definir el usuario-contexto, por ejemplo, en un canal de TDT, 4 Mbps, el problema del rate allocation se nos ha acabado si en el canal de mejora (el que pasa de la resolución "1" a la "0") también transporta 4 Mbps. Si tenemos más niveles de resolución, el problema se resuelve trivialmente configurando HEVC para producir code-streams de 4 Mbps/canal. Ahora ... dentro del canal de mejora hay que cuantificar 

Si lo que queremos es escalar en calidad, de nuevo, la primera capa de calidad de $L^0_0$ debería comprimirse a 4 Mbps.

## More resolutions
PRMC se puede aplicar de forma recursiva a la secuencia $L^1$, y luego a $L^2$, etc.

## Sobre este notebook

Este notebook está [aquí](https://github.com/vicente-gonzalez-ruiz/PRMC-experimentos).